In [2]:
import json
import time

# FILE_PATH = "data_pembelian_10000.json"
FILE_PATH = "data_dummy_pelanggan.json"

with open(FILE_PATH, 'r', encoding='utf-8') as file:
    data = json.load(file)

    len(data)

In [3]:
from operator import itemgetter


# Menambahkan field bulan_pembelian untuk memudahkan pencarian bulanan
for record in data:
    # Ekstrak bulan dan tahun dari tanggal_pembelian (format: dd-mm-yyyy)
    record["bulan_pembelian"] = record["tanggal_pembelian"][3:10]  # Format 'mm-yyyy'


# data yang perlu di search
data_by_id = sorted(data, key=itemgetter("id"))
data_by_nama = sorted(data, key=lambda r: r["nama_pelanggan"].lower() )
data_by_tanggal = sorted(data, key=itemgetter("tanggal_pembelian"))
data_by_jumlah = sorted(data, key=itemgetter("jumlah_pembelian"))
data_by_bulanan = sorted(data, key=itemgetter("bulan_pembelian"))

In [4]:
def binary_search(arr, target, keyfunc):
    """
    arr      : list of record
    target   : nilai yang dicari
    keyfunc  : fungsi untuk mengambil 'key' dari record
    return   : index salah satu elemen yang cocok, atau -1 jika tidak ketemu
    """
    left, right = 0, len(arr) - 1

    while left <= right:
        mid = (left + right) // 2
        k = keyfunc(arr[mid])

        if k == target:
            return mid
        elif k < target:
            left = mid + 1
        else:
            right = mid - 1

    return -1


In [5]:
def collect_all(arr, index, target, keyfunc):
    """
    Mengambil semua record yang key-nya == target di sekitar index hasil binary_search.
    """
    if index == -1:
        return []

    results = []

    # ke kiri
    i = index
    while i >= 0 and keyfunc(arr[i]) == target:
        results.append(arr[i])
        i -= 1
    results.reverse()  # supaya urutan dari kecil ke besar

    # ke kanan
    i = index + 1
    while i < len(arr) and keyfunc(arr[i]) == target:
        results.append(arr[i])
        i += 1

    return results


In [6]:
def cari_data():
    print("Jenis pencarian yang tersedia:")
    print("1. id")
    print("2. nama")
    print("3. tanggal")
    print("4. jumlah")
    print("5. bulanan")

    jenis = input("Masukkan jenis pencarian (id/nama/tanggal/jumlah/bulanan): ").strip().lower()

    if jenis == "id":
        target = input("Masukkan ID (misal: t0001): ").strip()
        arr = data_by_id
        keyfunc = lambda r: r["id"]

    elif jenis == "nama":
        target = input("Masukkan nama pelanggan: ").strip().lower()
        arr = data_by_nama
        keyfunc = lambda r: r["nama_pelanggan"].lower()

    elif jenis == "tanggal":
        # format harus sama dengan di file, misal: 06-03-2023
        target = input("Masukkan tanggal pembelian (dd-mm-yyyy): ").strip()
        arr = data_by_tanggal
        keyfunc = lambda r: r["tanggal_pembelian"]

    elif jenis == "bulanan":
        # format harus sama dengan di file, misal: 06-03-2023
        target = input("Masukkan bulanan pembelian (mm-yyyy): ").strip()
        arr = data_by_bulanan
        keyfunc = lambda r: r["bulan_pembelian"]

    elif jenis == "jumlah":
        target = int(input("Masukkan jumlah pembelian (angka): ").strip())
        arr = data_by_jumlah
        keyfunc = lambda r: r["jumlah_pembelian"]

    else:
        print("Jenis pencarian tidak dikenal.")
        return

    # ----- mulai ukur waktu pencarian -----
    start = time.perf_counter()
    idx = binary_search(arr, target, keyfunc)
    elapsed = (time.perf_counter() - start) * 1000  # ms
    # --------------------------------------

    if idx == -1:
        print(f"Data tidak ditemukan. Waktu pencarian: {elapsed:.6f} ms")
    else:
        results = collect_all(arr, idx, target, keyfunc)
        print(f"Ditemukan {len(results)} data. Waktu pencarian: {elapsed:.6f} ms")
        for i, r in enumerate(results, start=1):
            print(f"\nHasil ke-{i}:")
            print(f"  id               : {r['id']}")
            print(f"  nama_pelanggan   : {r['nama_pelanggan']}")
            print(f"  tanggal_pembelian: {r['tanggal_pembelian']}")
            print(f"  jumlah_pembelian : {r['jumlah_pembelian']}")
cari_data()

Jenis pencarian yang tersedia:
1. id
2. nama
3. tanggal
4. jumlah
5. bulanan
Ditemukan 1 data. Waktu pencarian: 0.017100 ms

Hasil ke-1:
  id               : t0001
  nama_pelanggan   : Bambang Fadli
  tanggal_pembelian: 06-03-2023
  jumlah_pembelian : 5
